# Machine learning applied to 0D reactors

In this notebook, we will train neural networks to replace the CVODE solver of CANTERA. We will use the databases generated in the *0D_database_generation.ipynb* notebook.

## Imports and options

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json
import joblib
import numpy as np
import pandas as pd

import cantera as ct

import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme("notebook")

from chem_ai.cantera_runs import compute_nn_cantera_0D_homo
from chem_ai.utils import get_molar_mass_atomic_matrix

We set the default pytorch precision to double. It slows down a little bit the training but it is the usual standard for CFD reacting flows applications.

In [ ]:
torch.set_default_dtype(torch.float64)

We identify the device (CPU or GPU) available on the machine. This will be used by pytorch to identify the device on which to train and use the model:

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda:0')
  print('Running on the GPU')
else:
  device = torch.device('cpu')
  print('Running on the CPU')

## Preliminary

### Loading the required data

We define the folder including the desired database:

In [ ]:
folder = "./case_0D_test_case" 

We load the parameters stored in the json file of the dabatase folder:

In [ ]:
with open(os.path.join(folder, "dtb_params.json"), "r") as file:
    dtb_params = json.load(file)

fuel = dtb_params["fuel"]
mech_file = dtb_params["mech_file"]
log_transform = dtb_params["log_transform"]
threshold = dtb_params["threshold"]
p = dtb_params["p"]
dt = dtb_params["dt"]

We load the scalers:

In [ ]:
Xscaler = joblib.load(os.path.join(folder, "processed_database", "Xscaler.pkl"))
Yscaler = joblib.load(os.path.join(folder, "processed_database", "Yscaler.pkl"))

We load the training and validation databases:

In [ ]:
X_train = pd.read_csv(os.path.join(folder, "processed_database","X_train.csv"))
X_val = pd.read_csv(os.path.join(folder, "processed_database","X_val.csv"))
Y_train = pd.read_csv(os.path.join(folder, "processed_database","Y_train.csv"))
Y_val = pd.read_csv(os.path.join(folder, "processed_database","Y_val.csv"))

Xcols = X_train.columns
Ycols = Y_train.columns

Number of input and output dimensions:

In [ ]:
n_in = X_train.shape[1]
n_out = Y_train.shape[1]

### Elements conservation matrix

In combustion, elements (usually C, H, O, N) are preserved when a mixture undergoes chemical reactions, as there are no nuclear reactions. Therefore, the initial mass of elements of a mixture is conserved at the next time step and so on. For a 0D reactors (no mixing), elements mass fractions are constant for a given simulation.

For a given element $j \in {C, H, O, N}$, the mass fraction of this elements can be expressed as:

$$
Y_e^j = \sum_{k=1}^{N_S} \frac{M_j}{M_k} n_k^j Y_k
$$

where $M_j$ and $M_k$ are the molar masses of element $j$ and species $k$ respectively. $n_k^j$ is the number of atoms $j$ in species $k$. This equation can also be written in matrix form:

$$
Y_e = A Y
$$

where $Y_e \in \mathbb{R}^4$ is the vector of elements mass fractions and $Y \in \mathbb{R}^{N_S}$ the vector of species mass fractions. The matrix $A \in \mathbb{R}^{4 \times N_S}$ is defined be the following coefficients:

$$
A_{jk} = \frac{M_j}{M_k} n_k^j
$$

The matrix $A$ can be computed using the function *get_molar_mass_atomic_matrix* given in *utils.py*:


In [ ]:
gas = ct.Solution(mech_file)
A_element = get_molar_mass_atomic_matrix(gas.species_names, fuel, True)
print(A_element)

This matrix will be helpful to analyze the conservation of elements in the training loop and at inference.

## Model definition and training

The data loaded (scalers, training/validation sets, etc...) are in numpy format. In order to use them in a Pytorch training loop, we need to convert them to *torch* tensors. Those tensors are very similar to numpy arrays, with similar functions.

We first transform training and validation datasets:

In [ ]:
X_train = torch.tensor(X_train.values, dtype=torch.float64)
Y_train = torch.tensor(Y_train.values, dtype=torch.float64)
X_val = torch.tensor(X_val.values, dtype=torch.float64)
Y_val = torch.tensor(Y_val.values, dtype=torch.float64)

In order to deal with the scaler, we decide to extract the mean and standard deviation and write the formula directly when necessary. These quantities are here converted to torch tensors:

In [ ]:
Xscaler_mean = torch.from_numpy(Xscaler.mean.values)
Xscaler_std = torch.from_numpy(Xscaler.std.values)

Yscaler_mean = torch.from_numpy(Yscaler.mean.values)
Yscaler_std = torch.from_numpy(Yscaler.std.values)

The conservation matrix $A$ also needs to be converted as it will be used during the training loop:

In [ ]:
A_element = torch.tensor(A_element, dtype=torch.float64)

Another aspect is that the data needs to be on the correct device, as pytorch will look for the data on it. As CPU and GPU memory is not shared, we will have to manually move the data to the GPU if necessary. We do it here for training/validation data, scalers and conservation matrix:

In [ ]:
X_train = X_train.to(device)
Y_train = Y_train.to(device)
X_val = X_val.to(device)
Y_val = Y_val.to(device)

Xscaler_mean = Xscaler_mean.to(device)
Xscaler_std = Xscaler_std.to(device)

Yscaler_mean = Yscaler_mean.to(device)
Yscaler_std = Yscaler_std.to(device)

A_element = A_element.to(device)

We now can generate the model. In this work, we will consider a simple Multi Layer Perceptron (MLP). We generate the model using Pytorch:

In [ ]:
class ChemNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(n_in, 60)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(60, 60)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(60, n_out)
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.output(x)
        return x

The model is then instantiated and transferred to the GPU if present:

In [ ]:
model = ChemNN()
print(model)

#Put model on GPU
model = model.to(device)

We now define hyperparameters of the training loop. Choices made here are standard. The following choices need to be made:

+ **n_epochs**: number of passies of entire training dataset through teh algorithm.
+ **batch_size**: size of the chunks passed to the algorithm at each parameters update.
+ **loss_fn**: loss function. In this we choose the Mean Square Loss (MSE), which is adapted to the regression problem. Assuming that the output of the ANN is $Y_k^n$ (preprocessed mass fractions) and the true value is $Y_k^{n,*}$, the loss reads:

$$ 
\mathcal{L} = \frac{1}{n} \sum_{i=1}^{n} \left( Y_k^n - Y_k^{n,*} \right)^2
$$

+ **optimizer**: optimization method. We use here the standard Adam method with initial learnign rate $lr$.

In [ ]:
n_epochs = 300
batch_size = 256

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

We perform now the main model training loop.

*Exercice 1:*

We give the following hints:

In [ ]:
# Array to store the loss and validation loss
loss_list = np.empty(n_epochs)
val_loss_list = np.empty(n_epochs//10)

# Array to store sum of mass fractions: mean, min and max
mean_sum_yk = np.empty(n_epochs//10)
max_sum_yk = np.empty(n_epochs//10)
min_sum_yk = np.empty(n_epochs//10)

# Array to store elements conservation: mean, min and max
mean_A_elements = np.empty((n_epochs//10,4))
max_A_elements = np.empty((n_epochs//10,4))
min_A_elements = np.empty((n_epochs//10,4))

epochs = np.arange(n_epochs)
epochs_small = epochs[::10]

for epoch in range(n_epochs):

    # Training parameters
    for i in range(0, len(X_train), batch_size):

        Xbatch = X_train[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = Y_train[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    loss_list[epoch] = loss

    # Computing validation loss and mass conservation metric (only every 10 epochs as it is expensive)
    if epoch%10==0:
        model.eval()
        with torch.no_grad():

            # Validation loss
            y_val_pred = model(X_val)
            val_loss = loss_fn(y_val_pred, Y_val)

            # Sum of mass fractions
            #Inverse scale done by hand to stay with Torch arrays
            yk = Yscaler_mean + (Yscaler_std + 1e-7)*y_val_pred
            if log_transform:
                yk = torch.exp(yk)
            sum_yk = yk.sum(axis=1)
            sum_yk = sum_yk.detach().cpu().numpy()
            mean_sum_yk[epoch//10] = sum_yk.mean()
            max_sum_yk[epoch//10] = sum_yk.max()
            min_sum_yk[epoch//10] = sum_yk.min()

            # Elements conservation
            # De-transforming inputs also
            yval_in = Xscaler_mean[1:] + (Xscaler_std[1:] + 1e-7)*X_val[:,1:]
            if log_transform:
                yval_in = torch.exp(yval_in)
            ye_in = torch.matmul(A_element, torch.transpose(yval_in, 0, 1))
            ye_out = torch.matmul(A_element, torch.transpose(yk, 0, 1))
            delta_ye = ye_out - ye_in
            delta_ye = delta_ye.detach().cpu().numpy()
            mean_A_elements[epoch//10, :] = delta_ye.mean(axis=1)
            min_A_elements[epoch//10, :] = delta_ye.min(axis=1)
            max_A_elements[epoch//10, :] = delta_ye.max(axis=1)

        model.train()
        val_loss_list[epoch//10] = val_loss

    print(f"Finished epoch {epoch}")
    print(f"    >> Loss: {loss}")
    if epoch%10==0:
        print(f"    >> Validation loss: {val_loss}")

We can plot the training and validation losses:

In [ ]:
fig, ax = plt.subplots()

ax.plot(epochs, loss_list, color="k", label="Training")
ax.plot(epochs_small, val_loss_list, color="r", label = "Validation")

ax.set_yscale('log')

ax.legend()

ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")

We plot the evolution of $\sum_{k=1}^{N_S} Y_k$ over epochs:

In [ ]:
fig, ax = plt.subplots()

ax.plot(epochs_small, mean_sum_yk, color="k")
ax.plot(epochs_small, max_sum_yk, color="k", ls="--")
ax.plot(epochs_small, min_sum_yk, color="k", ls="--")

ax.set_xlabel("Epoch")
ax.set_ylabel(r"$\sum_k \ Y_k$")

Analyzing conservation of chemical elements:

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)

# C
ax1.plot(epochs_small, mean_A_elements[:,0], color="k")
ax1.plot(epochs_small, min_A_elements[:,0], color="k", ls="--")
ax1.plot(epochs_small, max_A_elements[:,0], color="k", ls="--")

ax1.set_xlabel("Epoch")
ax1.set_ylabel(r"$\Delta Y_C$")

# H
ax2.plot(epochs_small, mean_A_elements[:,1], color="k")
ax2.plot(epochs_small, min_A_elements[:,1], color="k", ls="--")
ax2.plot(epochs_small, max_A_elements[:,1], color="k", ls="--")

ax2.set_xlabel("Epoch")
ax2.set_ylabel(r"$\Delta Y_H$")

# O
ax3.plot(epochs_small, mean_A_elements[:,2], color="k")
ax3.plot(epochs_small, min_A_elements[:,2], color="k", ls="--")
ax3.plot(epochs_small, max_A_elements[:,2], color="k", ls="--")

ax3.set_xlabel("Epoch")
ax3.set_ylabel(r"$\Delta Y_O$")

# N
ax4.plot(epochs_small, mean_A_elements[:,3], color="k")
ax4.plot(epochs_small, min_A_elements[:,3], color="k", ls="--")
ax4.plot(epochs_small, max_A_elements[:,3], color="k", ls="--")

ax4.set_xlabel("Epoch")
ax4.set_ylabel(r"$\Delta Y_N$")

fig.tight_layout()

We can finally save the Pytorch model in the case folder for later use:

In [ ]:
torch.save(model.state_dict(), os.path.join(folder,"pytorch_mlp.pt"))

## Testing the model

### Computing simulations with CVODE and NN

Getting the test simulations initial conditions:

In [ ]:
df_sim_test = pd.read_csv(os.path.join(folder, "sim_test.csv"))

n_sim = df_sim_test.shape[0]
print(f"There are {n_sim} test simulations")

In [ ]:
list_test_results = []

for i, row in df_sim_test.iterrows():

    phi_ini = row['Phi']
    temperature_ini = row['T0']

    print(f"Performing test computation for phi={phi_ini}; T0={temperature_ini}")

    df_exact, df_nn = compute_nn_cantera_0D_homo(device, model, Xscaler, Yscaler, phi_ini, temperature_ini, dt, dtb_params)

    list_test_results.append((df_exact, df_nn))

In [ ]:
i_sim = 40
df_exact = list_test_results[i_sim][0]
df_nn = list_test_results[i_sim][1]

# Temperature 
fig, ax = plt.subplots()

ax.plot(df_exact['Time'], df_exact['Temperature'], color='k')
ax.plot(df_nn['Time'], df_nn['Temperature'], color='b')


# Species
fig, ax = plt.subplots()

ax.plot(df_exact['Time'], df_exact['CH4'], color='k')
ax.plot(df_nn['Time'], df_nn['CH4'], color='b')


# Sum of Yk
fig, ax = plt.subplots()
ax.plot(df_nn['Time'], df_nn['SumYk'], color='b')


# Elements
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)
ax1.plot(df_nn['Time'], df_nn['Y_C'], color='b')
ax2.plot(df_nn['Time'], df_nn['Y_H'], color='b')
ax3.plot(df_nn['Time'], df_nn['Y_O'], color='b')
ax4.plot(df_nn['Time'], df_nn['Y_N'], color='b')
ax1.set_ylabel("$Y_C$")
ax2.set_ylabel("$Y_H$")
ax3.set_ylabel("$Y_O$")
ax4.set_ylabel("$Y_N$")
fig.tight_layout()

### Analyzing error statistics

We first need to re-scale and log transform. Loop on test simulations :

In [ ]:
data_errors = np.empty([n_sim, n_out+2])  # +2 because error on temperature is included, and mean of errors also

for i_sim in range(n_sim):

    df_exact = list_test_results[i_sim][0]
    df_nn = list_test_results[i_sim][1]

    # Removing undesired variables
    df_exact = df_exact.drop('Time', axis=1)
    df_nn = df_nn.drop(["Time","SumYk", "Y_C", "Y_H", "Y_O", "Y_N"], axis=1)


    # Applying log
    if log_transform:

        df_exact[df_exact < threshold] = threshold
        df_nn[df_nn < threshold] = threshold

        df_exact.iloc[:, 1:] = np.log(df_exact.iloc[:, 1:])
        df_nn.iloc[:, 1:] = np.log(df_nn.iloc[:, 1:])

    # Scaling
    data_exact_scaled = (df_exact.values-Xscaler.mean.values)/(Xscaler.std.values+1.0e-7)
    data_nn_scaled = (df_nn.values-Xscaler.mean.values)/(Xscaler.std.values+1.0e-7)

    diff_exact_nn = np.abs((data_nn_scaled-data_exact_scaled)/data_exact_scaled)

    diff_exact_nn = diff_exact_nn.mean(axis=0)

    M = diff_exact_nn.mean()

    print(f"Simulation {i_sim} error M = {M}")

    data_errors[i_sim, :n_out+1] = diff_exact_nn
    data_errors[i_sim, n_out+1] = diff_exact_nn.mean()
    

Computing M:

In [ ]:
# Getting gas species for labels
gas = ct.Solution(mech_file)


fig, ax = plt.subplots()

sns.boxplot(data_errors, ax=ax)

custom_labels = ["T"] + gas.species_names + ["Total"]
ax.set_xticklabels(custom_labels)


Global errors:

In [ ]:
M_vect = data_errors[:,-1]

print(f"Averaged on set of test simulations, error is M={M_vect.mean()} %")

Identifying simulation with the largest error:

In [ ]:
i_sim_max = M_vect.argmax()
print(f"Simulation with largest error: {i_sim_max}")
print(f"Error is: {M_vect[i_sim_max]} %")